<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://scontent.fpei1-1.fna.fbcdn.net/v/t39.30808-6/241998866_100151369093866_8422142037584747656_n.png?_nc_cat=105&ccb=1-7&_nc_sid=6ee11a&_nc_ohc=FchCmU1e1gEQ7kNvwE89YIg&_nc_oc=AdmQ9y67Ecx1BB_nvddzennfiXVsBDvNrPJ5hqlDinea5QAvxjZNtXsMhz8DDsDgyFy4H1vrFQA7trFxEm5y6Prv&_nc_zt=23&_nc_ht=scontent.fpei1-1.fna&_nc_gid=_d-S8PdbAXYSs7VxpPTPSw&oh=00_AfOBC8aP7NU5m3RyP6R5wA65f3f0nE-5y5EsdJgBLz-7Ww&oe=6864C04D"ASOCIO" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">STUDENT CHALLENGE'25: ASIGNACIÓN DE PUESTOS DE TRABAJO PARA UNA ESTRATEGÍA HIBRIDA DE PRESENCIALIDAD Y TELETRABAJO </strong><br><br> <span style="font-size: 30px;"> Valentina Jiménez Torres <br> Juan Camilo Henao Caro<br> Fernando Antonio Piñeres Ramos <br> <i>Estudiantes de ingeniería industrial <i>  <br> <i> Universidad de Antioquia <i> </span> </td> </tr> </table>




#### **DESCRIPCIÓN DEL PROBLEMA**

Desde 2022, la Dirección de Planeación y Desarrollo Institucional adoptó un **modelo de teletrabajo híbrido**, permitiendo a los colaboradores combinar el trabajo remoto con jornadas presenciales. **El cambio ha generado un nuevo reto logístico: la asignación de puestos de trabajo compartidos**. Actualmente, este proceso se realiza de forma manual, lo que consume mucho tiempo y dificulta la adaptación a cambios de último minuto, como ausencias inesperadas.

El objetivo es crear una **solución automatizada para optimizar la asignación de puestos de trabajo**. Para ello, se desarrolló un **modelo de optimización en Python** que distribuirá los espacios de manera equitativa y eficiente, considerando las necesidades de los colaboradores y la estructura de la Dirección.

#### 1. PREPARACIÓN DEL ENTORNO

##### **1.1 INSTALACIÓN E IMPORTE DE LIBRERÍAS**

In [103]:
#PuLP es una librería de python que se utiliza para modelar y resolver problemas de optimización, especialmente problemas de PL y PE.
!pip install --upgrade pulp

In [104]:
#Importe de librerías requeridas.
from google.colab import files #Permite seleccionar y subir archivos desde la computadora local a Colab.
import json # Permite trabajar con datos en formato JSON (JavaScript Object Notation).
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpStatus, LpBinary, PULP_CBC_CMD

Funciones Clave de PuLP
- LpProblem: Crea el modelo de optimización completo.
- LpVariable: Define las variables de decisión (ej. si un empleado va a un puesto).
- LpMaximize: Establece que el objetivo es maximizar un valor (ej. la satisfacción).
- lpSum: Permite hacer sumas eficientes para las restricciones y el objetivo.
- LpStatus: Verifica el estado de la solución (ej. si es óptima).
- LpBinary: Limita las variables a valores binarios (0 o 1), ideal para problemas de asignación.

















##### **1.2 LECTURA DE LAS INSTANCIAS DE DATOS**

In [105]:
uploaded = files.upload()
filename = next(iter(uploaded))

Saving instance10.json to instance10 (4).json


In [106]:
with open(filename) as f:
    data = json.load(f)

#### 2. FORMULACIÓN DEL MODELO DE ASIGNACIÓN

##### **2.1 DEFINICIÓN DE LOS CONJUNTOS Y PARÁMETROS**

In [107]:
# Conjuntos
E = data['Employees']  # Lista de empleados
D = data['Desks']      # Lista de escritorios
S = data['Days']       # Lista de días disponibles
G = data['Groups']     # Lista de grupos
Z = data['Zones']      # Lista de zonas físicas

# Parámetros
DesksZ_raw = data['Desks_Z'] #Escritorios por zona
DesksE = data['Desks_E']  #Escritorios por día
EmployeesG = data['Employees_G'] #Colaboradores por grupo
DaysE_orig = data['Days_E']  #Días preferidos por cada colaborador

In [108]:
# Se definen los días de presencialidad:
# Mínimo 2 días presenciales por empleado; un 3er día solo si hay espacio.

# Calcula el total de espacios disponibles en el sistema (escritorios × días).
total_slots = len(D) * len(S)

# Calcula el mínimo de asignaciones necesarias si cada empleado asiste al menos 2 días.
min_required = 2 * len(E)

# Calcula cuántas asignaciones "extra" hay disponibles después de cubrir el mínimo.
# Es decir, cuántos terceros días se pueden asignar si hay espacio físico.
max_extra = total_slots - min_required

In [109]:
# Se construye un diccionario que mapea cada escritorio a su zona correspondiente.
DesksZ = {}  # Diccionario vacío donde se guardará la zona de cada escritorio

# Itera por cada entrada en DesksZ_raw (clave: zona, valor: lista de escritorios)
for zonadesk, desks in DesksZ_raw.items():

    # Itera por cada escritorio en la lista de esa zona
    for d in desks:

        # Asocia el escritorio 'd' con su zona 'zonadesk' en el diccionario DesksZ
        DesksZ[d] = zonadesk

##### **2.2 DEFINICIÓN DE LAS VARIABLES DE DECISIÓN**


In [110]:
# Se crea el modelo de optimización llamado "Asignacion_Hibrida"
# El objetivo del modelo será MAXIMIZAR (LpMaximize) una función objetivo que definiremos más adelante.
model = LpProblem("Asignacion_Hibrida", LpMaximize)

In [111]:
# Variables de decisión:

# Variable binaria x[e][d][s]: 1 si el empleado 'e' es asignado al escritorio 'd' el día 's'; 0 en caso contrario.
# Esta variable representa la asignación concreta de escritorios.
x = {}
for e in E:
    for d in DesksE[e]:  # Solo escritorios compatibles
        for s in S:
            x[e, d, s] = LpVariable(f"x_{e}_{d}_{s}", cat=LpBinary)

In [112]:
# Variable binaria z[e][s]: 1 si el empleado 'e' asiste presencialmente el día 's'; 0 si trabaja remoto.
# Es útil para controlar cuántos días asiste cada empleado y si hay coherencia con la asignación de escritorios.
z = LpVariable.dicts("z", (E, S), cat=LpBinary)

In [113]:
# Variable binaria y[g][s]: 1 si el grupo 'g' tiene su reunión presencial el día 's'; 0 en caso contrario.
# Sirve para planificar los días de reunión por grupo.
y = LpVariable.dicts("y", (G, S), cat=LpBinary)

In [114]:
# Variable binaria w[g][z][s]: 1 si el grupo 'g' utiliza la zona 'z' el día 's'; 0 en caso contrario.
# Indica qué zonas se activan para un grupo en su día de reunión.
# w[g][z][s] solo si hay posibilidad de uso real, solo si hay escritorios en z compatibles con empleados de g
w = {}
for g in G:
    for s in S:
        for z_id in Z:
            empleados = EmployeesG[g]
            escritorios_en_zona = [d for d in D if DesksZ[d] == z_id]
            if any(d in DesksE[e] for e in empleados for d in escritorios_en_zona):
                w[g, z_id, s] = LpVariable(f"w_{g}_{z_id}_{s}", cat=LpBinary)

In [115]:
# Variable binaria u[g][e][s]: 1 si el empleado 'e' del grupo 'g' asiste presencialmente el día de reunión del grupo (z ∧ y); 0 si no.
# Esta variable modela la intersección lógica entre "el empleado asiste" y "es el día de reunión del grupo".
# u[g][e][s] solo si e pertenece al grupo g, solo para e ∈ EmployeesG[g]
u = {}
for g in G:
    for e in EmployeesG[g]:
        for s in S:
            u[g, e, s] = LpVariable(f"u_{g}_{e}_{s}", cat=LpBinary)

In [116]:
# Variable binaria extra[e]: 1 si al empleado 'e' se le asigna un tercer día presencial (adicional a los 2 mínimos); 0 si no.
# Permite controlar cuántos empleados reciben un día extra según la capacidad disponible.
extra = LpVariable.dicts("extra", E, cat=LpBinary)

##### **2.3 DEFINICIÓN DE RESTRICCIONES**


In [117]:
# R1: compatibilidad de escritorios
for e in E:
    for d in DesksE[e]:  # ¡Aquí ya filtras por compatibilidad!
        for s in S:
            x[e, d, s] = LpVariable(f"x_{e}_{d}_{s}", cat=LpBinary)
                # Si el escritorio 'd' no es compatible con el empleado 'e',
                # entonces no se le puede asignar ese escritorio el día 's'.

In [118]:
# R2: un escritorio por día por empleado
for e in E:
    for s in S:
        # Cada empleado puede estar asignado a lo sumo a 1 escritorio por día.
        model += lpSum(x.get((e, d, s), 0) for d in DesksE[e]) <= 1

In [119]:
# R3: un escritorio no se comparte
EmpDesk = {d: [e for e in E if d in DesksE[e]] for d in D} # empleados que pueden usar cada escritorio
for d in D:
    for s in S:
        model += lpSum(x.get((e, d, s), 0) for e in EmpDesk[d]) <= 1

In [120]:
# R4: asignar mínimo 2 días por empleado, un 3ro solo si hay espacio
for e in E:
    # Cada empleado debe asistir al menos 2 días presenciales.
    model += lpSum(z[e][s] for s in S) >= 2

    # Puede asistir un tercer día solo si extra[e] = 1.
    model += lpSum(z[e][s] for s in S) <= 2 + extra[e]

# El número total de asignaciones extra no puede superar la capacidad disponible.
model += lpSum(extra[e] for e in E) <= max_extra

In [121]:
# R5: cada grupo debe tener exactamente un día de reunión
for g in G:
    # Cada grupo tiene un día de reunión en la semana.
    model += lpSum(y[g][s] for s in S) == 1

In [122]:
# R6: activación de zonas por grupo para reunión.
# Solo permitir w[g][z][s] = 1 si hay reunión y alguien del grupo se sienta allí
for g in G:
    for s in S:
        for z_id in Z:
            empleados = EmployeesG[g]
            escritorios_en_zona = [d for d in D if DesksZ[d] == z_id]

            # Si w[g][z][s] = 1 (la zona está activa), entonces debe haber alguien del grupo allí sentado.
            model += lpSum(x.get((e, d, s), 0) for e in empleados for d in escritorios_en_zona) <= len(escritorios_en_zona) * w.get((g, z_id, s), 0)

            # Y viceversa: si alguien del grupo se sienta allí, la zona debe estar activa.
            model += lpSum(x.get((e, d, s), 0) for e in empleados for d in escritorios_en_zona) >= w.get((g, z_id, s), 0)

In [123]:
# R7: coherencia entre presencia y asignación de escritorio
for e in E:
    for s in S:
        # Si el empleado está presente el día 's', debe estar asignado exactamente a un escritorio.
        model += lpSum(x.get((e, d, s), 0) for d in DesksE[e]) == z[e][s]

In [124]:
# R8: linealización del producto lógico u[g,e,s] = z[e,s] ∧ y[g,s]
#u[g,e,s] = 1 si el empleado 'e' del grupo 'g' está presente el día 's' (z[e][s] = 1) Y ese día es el de reunión del grupo 'g' (y[g][s] = 1).
for g in G:
    for s in S:
        for e in EmployeesG[g]:
            # u[g,e,s] solo puede valer 1 si el empleado 'e' asiste ese día (z=1) y hay reunión del grupo (y=1).
            model += u.get((g, e, s), 0) <= z[e][s]
            model += u.get((g, e, s), 0) <= y[g][s]
            model += u.get((g, e, s), 0) >= z[e][s] + y[g][s] - 1  # Inecuación que fuerza el AND lógico

##### **2.4 DEFINICIÓN DE LA FUNCIÓN OBJETIVO**


In [125]:
alpha_orig = 2.0   # Recompensa por asignar un día que el empleado realmente prefiere.
alpha_fill = 0.5   # Recompensa menor por asignar un día que NO está entre los preferidos.
beta       = 1.2   # Recompensa por programar una reunión grupal (por cada grupo).
gamma      = 0.5   # Penalización por usar más de una zona para un grupo en su reunión.
theta      = 1.0   # Recompensa por cada miembro del grupo que asiste el día de la reunión.

In [126]:
# Suma ponderada de días preferidos asignados a cada empleado.
orig_term = lpSum(
    alpha_orig * z[e][s]
    for e in E for s in DaysE_orig[e])

# Suma ponderada (menor) por asignar días que el empleado no eligió.
fill_term = lpSum(
    alpha_fill * z[e][s]
    for e in E for s in S if s not in DaysE_orig[e])

# Recompensa por cada reunión grupal planificada (una por grupo).
meet_term = beta * lpSum(y[g][s] for g in G for s in S)

# Recompensa por cada miembro que asiste presencialmente el día de reunión de su grupo.
cover_term = theta * lpSum(u.get((g, e, s), 0) for g in G for e in EmployeesG[g] for s in S)

# Penalización por cada zona que usa un grupo el día de su reunión.
# Idealmente, los grupos deberían estar todos en una sola zona.
zone_term = -gamma * lpSum(w.get((g, z_id, s), 0) for g in G for z_id in Z for s in S)

# Pequeña recompensa por permitir que ciertos empleados asistan un día extra (3 días).
bonus_term = 0.1 * lpSum(extra[e] for e in E)

In [127]:
# Función objetivo completa
# Se maximiza la suma total de beneficios y penalizaciones definidas anteriormente.
model += orig_term + fill_term + meet_term + cover_term + zone_term + bonus_term

#### 3. RESULTADOS DEL MODELO

In [128]:
# Número total de variables de decisión
num_vars = len(model.variables())
# Número total de restricciones
num_cons = len(model.constraints)

print(f" Número de variables: {num_vars}")
print(f" Número de restricciones: {num_cons}")

 Número de variables: 17465
 Número de restricciones: 5644


In [130]:
# Usamos 4 hilos y permitimos un gap relativo del 1%
solver = PULP_CBC_CMD(msg=True, threads=4, gapRel=0.01,timeLimit=2000)

model.solve(solver)
print("Estado:", LpStatus[model.status])

Estado: Optimal


In [ ]:
# model.solve()
# print("Estado:", LpStatus[model.status], "\n")

In [ ]:
# Se plantea este solver para revisar en las instancias mas grandes en un periodo de tiempo limitado la mejor solución
# from pulp import PULP_CBC_CMD
# model.solve(PULP_CBC_CMD(timeLimit=600, msg=True))
# print("Estado:", LpStatus[model.status], "\n")

In [ ]:
# !pip install mip
# import mip

# m = mip.Model(sense=mip.MAXIMIZE)  # reescribe tu modelo en la API de python‑mip
# # ... crea variables, restricciones y objetivo ...
# m.optimize(max_threads=4, mip_gap=0.01)  # multicore + gapRel
# print(m.status)

In [131]:
# Mostrar asignaciones completas por empleado
print("\nAsignaciones completas:")
for e in E:
    print(f"\n🔹 Empleado {e}")
    assigned_days = 0
    for s in S:
        if z[e][s].varValue == 1:
            assigned_days += 1
            # Busca el escritorio asignado (solo entre los compatibles)
            desk = next(
                (d for d in DesksE[e] if x[(e, d, s)].varValue == 1),
                None)
            zone = DesksZ.get(desk, "N/A")
            print(f" ▸ Día {s} — Escritorio {desk} — Zona {zone}")
    print(f"  Total días asignados: {assigned_days}")



Asignaciones completas:

🔹 Empleado E0
 ▸ Día Ma — Escritorio D7 — Zona Z2
 ▸ Día J — Escritorio D24 — Zona Z8
 ▸ Día V — Escritorio D29 — Zona Z9
  Total días asignados: 3

🔹 Empleado E1
 ▸ Día Ma — Escritorio D6 — Zona Z2
 ▸ Día Mi — Escritorio D28 — Zona Z9
 ▸ Día J — Escritorio D12 — Zona Z4
  Total días asignados: 3

🔹 Empleado E2
 ▸ Día L — Escritorio D32 — Zona Z10
 ▸ Día J — Escritorio D26 — Zona Z8
  Total días asignados: 2

🔹 Empleado E3
 ▸ Día Ma — Escritorio D8 — Zona Z2
 ▸ Día Mi — Escritorio D27 — Zona Z9
 ▸ Día J — Escritorio D25 — Zona Z8
  Total días asignados: 3

🔹 Empleado E4
 ▸ Día L — Escritorio D31 — Zona Z10
 ▸ Día J — Escritorio D14 — Zona Z4
  Total días asignados: 2

🔹 Empleado E5
 ▸ Día Mi — Escritorio D29 — Zona Z9
 ▸ Día V — Escritorio D7 — Zona Z2
  Total días asignados: 2

🔹 Empleado E6
 ▸ Día L — Escritorio D9 — Zona Z3
 ▸ Día J — Escritorio D43 — Zona Z14
 ▸ Día V — Escritorio D22 — Zona Z7
  Total días asignados: 3

🔹 Empleado E7
 ▸ Día J — Escritorio

In [132]:
# Mostrar zonas activadas y empleados presentes por grupo en su día de reunión
print("\nZonas activadas y asistencia por grupo en su día de reunión:\n")
for g in G:
    for s in S:
        if y[g][s].varValue == 1:
            # Usamos .get para evitar KeyError si w[g,z,s] no fue definida
            zonas_activas = [
                z_id for z_id in Z if w.get((g, z_id, s), None) and w[g, z_id, s].varValue == 1
            ]
            empleados_presentes = [
                e for e in EmployeesG[g] if z[e][s].varValue == 1
            ]
            print(f"🔹 Grupo {g} — Día de reunión: {s}")
            print(f"  ▸ Zonas activadas: {', '.join(zonas_activas) if zonas_activas else 'Ninguna'}")
            print(f"  ▸ Empleados presentes: {', '.join(empleados_presentes) if empleados_presentes else 'Ninguno'}\n")


Zonas activadas y asistencia por grupo en su día de reunión:

🔹 Grupo G0 — Día de reunión: J
  ▸ Zonas activadas: Z4, Z8
  ▸ Empleados presentes: E0, E1, E2, E3, E4

🔹 Grupo G1 — Día de reunión: J
  ▸ Zonas activadas: Z10, Z14
  ▸ Empleados presentes: E6, E7, E9, E11

🔹 Grupo G2 — Día de reunión: J
  ▸ Zonas activadas: Z3, Z12
  ▸ Empleados presentes: E12, E13, E14, E15, E16, E17

🔹 Grupo G3 — Día de reunión: L
  ▸ Zonas activadas: Z6, Z9
  ▸ Empleados presentes: E18, E19, E20, E21, E23

🔹 Grupo G4 — Día de reunión: V
  ▸ Zonas activadas: Z3, Z4
  ▸ Empleados presentes: E24, E25, E27, E28, E29

🔹 Grupo G5 — Día de reunión: L
  ▸ Zonas activadas: Z1, Z12
  ▸ Empleados presentes: E30, E32, E33, E34

🔹 Grupo G6 — Día de reunión: V
  ▸ Zonas activadas: Z8
  ▸ Empleados presentes: E37, E39, E41

🔹 Grupo G7 — Día de reunión: Mi
  ▸ Zonas activadas: Z2, Z11
  ▸ Empleados presentes: E42, E43, E44, E45, E46, E47

🔹 Grupo G8 — Día de reunión: V
  ▸ Zonas activadas: Z7, Z10
  ▸ Empleados present

In [133]:
# Mostrar resumen diario de todas las asignaciones
print("\nResumen diario de asignaciones:\n")
for s in S:  # Para cada día
    empleados_dia = []      # Lista de empleados asignados ese día
    escritorios_dia = []    # Lista de escritorios usados ese día
    zonas_dia = []          # Lista de zonas en las que se ubican esos escritorios
    for e in E:
        for d in D:
            var = x.get((e, d, s), None)
            if var and var.varValue == 1:  # Verifica que la variable existe y fue activada
                empleados_dia.append(e)
                escritorios_dia.append(d)
                zonas_dia.append(DesksZ[d])
    print(f"📅 Día {s}")
    print(f"  ▸ Total empleados asignados: {len(empleados_dia)}")
    if empleados_dia:
        for e, d, z in zip(empleados_dia, escritorios_dia, zonas_dia):
            print(f"    - Empleado {e} → Escritorio {d} (Zona {z})")
    else:
        print("    - No hay asignaciones.")
    print("-" * 40)



Resumen diario de asignaciones:

📅 Día L
  ▸ Total empleados asignados: 45
    - Empleado E2 → Escritorio D32 (Zona Z10)
    - Empleado E4 → Escritorio D31 (Zona Z10)
    - Empleado E6 → Escritorio D9 (Zona Z3)
    - Empleado E10 → Escritorio D10 (Zona Z3)
    - Empleado E18 → Escritorio D20 (Zona Z6)
    - Empleado E19 → Escritorio D29 (Zona Z9)
    - Empleado E20 → Escritorio D27 (Zona Z9)
    - Empleado E21 → Escritorio D19 (Zona Z6)
    - Empleado E23 → Escritorio D28 (Zona Z9)
    - Empleado E25 → Escritorio D39 (Zona Z13)
    - Empleado E26 → Escritorio D40 (Zona Z13)
    - Empleado E27 → Escritorio D38 (Zona Z12)
    - Empleado E30 → Escritorio D36 (Zona Z12)
    - Empleado E32 → Escritorio D5 (Zona Z1)
    - Empleado E33 → Escritorio D4 (Zona Z1)
    - Empleado E34 → Escritorio D37 (Zona Z12)
    - Empleado E36 → Escritorio D2 (Zona Z0)
    - Empleado E38 → Escritorio D0 (Zona Z0)
    - Empleado E40 → Escritorio D1 (Zona Z0)
    - Empleado E45 → Escritorio D16 (Zona Z5)
    - 

#### 4. MÉTRICAS

In [134]:
# A. Satisfacción con días preferidos
# Inicializamos contadores para las métricas de satisfacción
preferidos_totales = 0                     # Número total de días asignados que coinciden con los días preferidos
asignaciones_totales = 0                   # Número total de días presenciales asignados a todos los empleados
empleados_completamente_satisfechos = 0    # Número de empleados que recibieron SOLO días preferidos
# Recorremos todos los empleados
for e in E:
    dias_asignados = []  # Lista para guardar los días que fueron asignados al empleado e
    # Recorremos todas las variables del modelo
    for v in model.variables():
        # Identificamos las variables z[e][s] que indican presencia de un empleado e en un día s
        if v.name.startswith(f"z_{e}_") and v.varValue == 1:
            s = v.name.split("_")[2]  # Extraemos el día (s) del nombre de la variable
            dias_asignados.append(s)  # Agregamos ese día a la lista de asignaciones del empleado
    # Obtenemos los días preferidos por el empleado e desde los datos originales
    dias_preferidos = set(DaysE_orig.get(e, []))
    # Filtramos cuántos de los días asignados están dentro de los preferidos
    preferidos = [s for s in dias_asignados if s in dias_preferidos]
    # Acumulamos los totales
    preferidos_totales += len(preferidos)
    asignaciones_totales += len(dias_asignados)
    # Si TODOS los días asignados están dentro de los días preferidos, sumamos 1 al contador
    if dias_asignados and set(dias_asignados).issubset(dias_preferidos):
        empleados_completamente_satisfechos += 1
# Si hubo asignaciones, mostramos las métricas calculadas
if asignaciones_totales > 0:
    print(f" % de días asignados que son preferidos: {100 * preferidos_totales / asignaciones_totales:.2f}%")
    print(f" % de empleados con todos los días preferidos asignados : {100 * empleados_completamente_satisfechos / len(E):.2f}%")

 % de días asignados que son preferidos: 96.44%
 % de empleados con todos los días preferidos asignados : 92.00%


In [135]:
# B. Cohesión de grupo: % de miembros presentes en su día de reunión
asistencias = 0  # Contador de asistencias efectivas de empleados en el día de reunión de su grupo
posibles = 0     # Total de posibles asistencias (empleados por grupo por su día de reunión)
# Recorremos cada grupo g
for g in G:
    for s in S:
        # Verificamos si ese día s es el día de reunión del grupo g (variable y[g][s] == 1)
        if any(v.name == f"y_{g}_{s}" and v.varValue == 1 for v in model.variables()):
            for e in EmployeesG[g]:  # Recorremos los empleados del grupo
                posibles += 1  # Este empleado debería asistir ese día
                var_name = f"u_{g}_{e}_{s}"  # u[g][e][s] = 1 si el empleado estuvo presente y hubo reunión
                if any(v.name == var_name and v.varValue == 1 for v in model.variables()):
                    asistencias += 1  # Si efectivamente asistió, incrementamos asistencias

# Calculamos y mostramos el porcentaje de asistencia grupal en el día de reunión
if posibles > 0:
    print(f" % de asistencia presencial en día de reunión del grupo: {100 * asistencias / posibles:.2f}%")

 % de asistencia presencial en día de reunión del grupo: 79.00%


In [136]:
# C. Promedio de zonas distintas usadas por grupo en su día de reunión
zonas_por_grupo = []  # Lista para guardar cuántas zonas activó cada grupo el día de su reunión
# Recorremos cada grupo g
for g in G:
    for s in S:
        # Verificamos si ese día s es el día de reunión del grupo g
        if any(v.name == f"y_{g}_{s}" and v.varValue == 1 for v in model.variables()):
            zonas_activadas = 0  # Contador de zonas activadas por ese grupo en ese día

            # Recorremos todas las zonas
            for z_id in Z:
                var_name = f"w_{g}_{z_id}_{s}"  # w[g][z][s] indica si el grupo g activó la zona z ese día
                if any(v.name == var_name and v.varValue == 1 for v in model.variables()):
                    zonas_activadas += 1  # Se suma si la zona fue efectivamente usada
            zonas_por_grupo.append(zonas_activadas)  # Guardamos cuántas zonas usó el grupo g ese día
# Calculamos y mostramos el promedio de zonas distintas por grupo en su día de reunión
if zonas_por_grupo:
    promedio = sum(zonas_por_grupo) / len(zonas_por_grupo)
    print(f" Promedio de zonas distintas usadas por grupo en su día de reunión: {promedio:.2f}")

 Promedio de zonas distintas usadas por grupo en su día de reunión: 1.83
